In [31]:
import os
os.chdir("../data/raw")
os.getcwd()

FileNotFoundError: [Errno 2] No such file or directory: '../data/raw'

In [73]:
import pandas as pd
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit, cross_val_score, cross_validate
from sklearn.dummy import DummyRegressor

In [61]:
air_visit_data = pd.read_csv("air_visit_data.csv")
air_visit_data['visit_date'] = pd.to_datetime(air_visit_data['visit_date'])

In [48]:
data = air_visit_data.where(air_visit_data["air_store_id"] == "air_ba937bf13d40fb24").dropna().reset_index(drop = True)
data = data.drop(columns=["air_store_id"])
data

,visit_date,visitors
0,2016-01-13,25.0
1,2016-01-14,32.0
2,2016-01-15,29.0
3,2016-01-16,22.0
4,2016-01-18,6.0
...,...,...
386,2017-04-18,11.0
387,2017-04-19,11.0
388,2017-04-20,14.0
389,2017-04-21,40.0


In [102]:
X = data[["visit_date"]]
y = data[["visitors"]]

In [107]:
def cross_val(estimator, X, y, p_grid, inner_splits=5, outer_splits=5, inner_gap=3, outer_gap=3, test_size = 30):
    inner_cv = TimeSeriesSplit(n_splits=inner_splits, gap=inner_gap, test_size=test_size)
    outer_cv = TimeSeriesSplit(n_splits=outer_splits, gap=outer_gap, test_size=test_size)

    model = GridSearchCV(estimator=estimator, 
                         param_grid=p_grid, 
                         cv=inner_cv, 
                         scoring='neg_root_mean_squared_error')
    
    cv_results = cross_validate(model, 
                                X=X, 
                                y=y, 
                                cv=outer_cv, 
                                scoring='neg_root_mean_squared_error',
                                return_estimator=True)
    
    
    # print(cv_results['test_score'].mean())
    return cv_results
    # print(cv_results)

In [127]:
from sklearn import svm

reg = svm.SVR()
p_grid = {"C": [1, 10], "gamma": [0.01, 0.1]}

reg = DummyRegressor()

p_grid = {"strategy": ["mean", "median"]}

reg = cross_val(reg, X, y, p_grid)
# reg.predict(X)
print(reg['estimator'][0].best_estimator_.get_params())
print(reg['estimator'][1].best_estimator_.get_params())
print(reg['estimator'][2].best_estimator_.get_params())
print(reg['estimator'][3].best_estimator_.get_params())
print(reg['estimator'][4].best_estimator_.get_params())
reg


{'constant': None, 'quantile': None, 'strategy': 'mean'}
{'constant': None, 'quantile': None, 'strategy': 'mean'}
{'constant': None, 'quantile': None, 'strategy': 'median'}
{'constant': None, 'quantile': None, 'strategy': 'median'}
{'constant': None, 'quantile': None, 'strategy': 'median'}


{'fit_time': array([0.09828091, 0.07845545, 0.05742097, 0.05319786, 0.04924345]),
 'score_time': array([0.00168228, 0.00254846, 0.00217724, 0.00137591, 0.00173116]),
 'estimator': [GridSearchCV(cv=TimeSeriesSplit(gap=3, max_train_size=None, n_splits=5, test_size=30),
               estimator=DummyRegressor(),
               param_grid={'strategy': ['mean', 'median']},
               scoring='neg_root_mean_squared_error'),
  GridSearchCV(cv=TimeSeriesSplit(gap=3, max_train_size=None, n_splits=5, test_size=30),
               estimator=DummyRegressor(),
               param_grid={'strategy': ['mean', 'median']},
               scoring='neg_root_mean_squared_error'),
  GridSearchCV(cv=TimeSeriesSplit(gap=3, max_train_size=None, n_splits=5, test_size=30),
               estimator=DummyRegressor(),
               param_grid={'strategy': ['mean', 'median']},
               scoring='neg_root_mean_squared_error'),
  GridSearchCV(cv=TimeSeriesSplit(gap=3, max_train_size=None, n_splits=5, test_s